[View in Colaboratory](https://colab.research.google.com/github/mingweihe/HandGestureRecognition/blob/master/RawToTfRecords.ipynb)

In [0]:
!rm -rf data && rm -rf checkpoints
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from IPython.display import clear_output

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data/gesture')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1wX6r6kcXJqYigK6UPq35RDICKRL_pZ8M' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  
!unzip /content/data/gesture/*.zip -d /content/data/gesture/
!mkdir checkpoints
clear_output()

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
!mkdir Records

In [0]:
# set parameter
batch_size = 1
image_height = 300
image_width = 370
num_channels = 3
NUM_CLASS = 6

In [0]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def queuingToTfRecord(fromDirs, toDirs, metaDirs):
    for index in range(len(fromDirs)):
        fromDir, toDir = fromDirs[index], toDirs[index]
        writer=tf.python_io.TFRecordWriter(toDir)
        fNamePos = len(fromDir.split(os.path.sep))
        pattern = os.path.join(fromDir, '*.png')
        filenames = tf.train.match_filenames_once(pattern, name='list_files')
        queue = tf.train.string_input_producer(filenames, num_epochs = 1, shuffle=True, name='queue')
        reader = tf.WholeFileReader()
        filename, content = reader.read(queue, name='read_image')
        filename = tf.Print(filename, data=[filename], message='loading: ')
        filename_split = tf.string_split([filename], delimiter=os.path.sep)
        label = tf.string_to_number(tf.substr(filename_split.values[fNamePos], 0, 1), out_type=tf.int32)
        img_tensor = tf.image.decode_png(content, dtype=tf.uint8, channels=3, name='img_decode')
        min_after_dequeue = 1000
        capacity = min_after_dequeue + 3 * batch_size
        image_batch,label_batch=tf.train.shuffle_batch([img_tensor,label],batch_size=batch_size,
        shapes=[(image_height, image_width, num_channels), ()],capacity=capacity,
        min_after_dequeue=min_after_dequeue, name='train_shuffle')
        # Initialize variables
        init = (tf.global_variables_initializer(), tf.local_variables_initializer())
        # Launch session
        sess = tf.Session()
        sess.run(init)
        coord=tf.train.Coordinator()
        threads=tf.train.start_queue_runners(sess=sess,coord=coord)
        
        try:
            cnt = 0
            while not coord.should_stop():
                cur_image_batch,cur_label_batch = sess.run([image_batch,label_batch])
                for i in range(len(cur_label_batch)):
                    image_raw=cur_image_batch[i].tostring()
                    example=tf.train.Example(features=tf.train.Features(feature={
                    'label':_int64_feature(cur_label_batch[i]),'image_raw':_bytes_feature(image_raw), }))
                    writer.write(example.SerializeToString())
                    cnt += 1
        except tf.errors.OutOfRangeError:
            metaWriter = tf.python_io.TFRecordWriter(metaDirs[index])
            example=tf.train.Example(features=tf.train.Features(feature={'size':_int64_feature(cnt)}))
            metaWriter.write(example.SerializeToString())
            metaWriter.close()
            print('Done.')
        except Exception as e:
            coord.request_stop(e)
        finally:
            coord.request_stop()
        coord.join(threads)
        print(cnt, 'TFRecords have been generated.') 
        sess.close()
        writer.close()


In [5]:
fromDirs = [os.path.join('data', 'gesture', 'training_set'), os.path.join('data', 'gesture', 'test_set')]
toDirs= [os.path.join('Records', 'train.tfrecords'), os.path.join('Records', 'test.tfrecords')]
metaDirs = [os.path.join('Records', 'train.meta'), os.path.join('Records', 'test.meta')]
queuingToTfRecord(fromDirs, toDirs, metaDirs)

Done.
12000 TFRecords have been generated.
Done.
369 TFRecords have been generated.
